# Evaluación de Hardware: Loihi vs CPU

**Descripción:** Tutorial interactivo about the biological learning mechanism STDP (Spike-Timing-Dependent Plasticity) used in neuromórfico neural networks. Demonstrates how neurons learn temporal correlations automáticamente.

**Autor:** Mauro Risonho de Paula Assumpção.
**Fecha de creación:** December 5, 2025.
**Licencia:** MIT License.
**Desarrollo:** Human + AI Assisted Development (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

## Objetivo

Compare the performance of fraud detection implementation with SNN in:
- **CPU Tradicional** (Brian2 yesulator)
- **Intel Loihi 2** (yesulation of hardware neuromórfico)

## Métricas evaluadas

1. **latencia** (ms per inference)
2. **Rendimiento** (transactions by per)
3. **energía** (millijoules)
4. **potencia** (milliwatts)
5. **eficiencia** (speedup e potencia eficiencia)

In [ ]:
import sys
from pathlib import Path

# Determine the project root directory
# The notebook is in: portfolio/01_fraud_neuromórfico/notebooks/
# We need to reach: portfolio/01_fraud_neuromórfico/
notebook_dir = Path.cwd()
if 'notebooks' in str(notebook_dir):
    # If we are in .../portfolio/01_fraud_neuromórfico/notebooks
    project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
elif '01_fraud_neuromórfico' not in str(notebook_dir):
    # If we are in repository root, navigate to the project
    project_root = notebook_dir / 'portfolio' / '01_fraud_neuromórfico'
else:
    # Already in the project root directory
    project_root = notebook_dir

src_path = project_root / 'src'
hardware_path = project_root / 'hardware'

# Remove previous paths if they exist to avoid duplicates
for path in [str(src_path), str(hardware_path)]:
    if path in sys.path:
        sys.path.remove(path)

# Add to the start of path
sys.path.insert(0, str(src_path))
sys.path.insert(0, str(hardware_path))

# Verificar if the directories exist
print(f" Current directory: {notebook_dir}")
print(f" Project root: {project_root}")
print(f" Src path exists: {src_path.exists()}")
print(f" Hardware path exists: {hardware_path.exists()}")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm.auto import tqdm
import brian2

# Configurar Brian2 to use numpy (avoids errors from compilation C++ and problems with SymPy/Cython)
# Isso also suprime os logs of compilation of Cython que você estava vendo
brian2.prefs.codegen.target = "numpy"

# Importaciones of the project - directly since already in sys.path
from main import FraudDetectionPipeline, generate_synthetic_transactions
from loihi_simulator import LoihiSimulator, compare_with_cpu, LoihiSpecs # type: ignore

# Configurar visualization
plt.style.use('seaborn-v0_8-darkgrid')

## 1. CPU Evaluación

First, let's measure the real performance of the implementation running on CPU.

In [ ]:
# Generar test dataset
print(" Generando test dataset...")
df_train = generate_synthetic_transactions(n=500, fraud_ratio=0.2)
df_test = generate_synthetic_transactions(n=1000, fraud_ratio=0.2)

# Separar features e labels
feature_cols = ['amount', 'daily_frequency']
X_train = df_train[feature_cols].values
y_train = df_train['is_fraud'].values
X_test = df_test[feature_cols].values
y_test = df_test['is_fraud'].values

print(f"Train: X={X_train.shape}, y={y_train.shape}")
print(f"Test: X={X_test.shape}, y={y_test.shape}")

# Converter for numpy array explicitamente for evitar errors of tipagem of Pylance
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

print(f"Fraudes no treino: {int(np.sum(y_train_np))}/{len(y_train_np)} ({float(np.mean(y_train_np))*100:.1f}%)")
print(f"Fraudes no Test: {int(np.sum(y_test_np))}/{len(y_test_np)} ({float(np.mean(y_test_np))*100:.1f}%)")

In [ ]:
# Initializar model (usar without training for inference benchmark)
print(" Initializando pipeline SNN...")
pipeline = FraudDetectionPipeline()

print(" Pipeline initialized")
print(" Nota: This benchmark focuses on INFERENCE latencia")
print(" The model is using random weights - in production would be pre-trained")

In [ ]:
# Important: Reload modules after corrections
import importlib
import sys

# Reload project modules
if 'models_snn' in sys.modules:
 importlib.reload(sys.modules['models_snn'])
if 'encoders' in sys.modules:
 importlib.reload(sys.modules['encoders'])
if 'main' in sys.modules:
 importlib.reload(sys.modules['main'])

# Reimport after reload
from main import FraudDetectionPipeline, generate_synthetic_transactions

print("✅ Modules reloaded with dt=0.1ms corrections")

In [ ]:
# Reinitialize pipeline with corrected modules
print("⚙️ Reinitializing SNN pipeline with dt=0.1ms...")
pipeline = FraudDetectionPipeline()

print("✅ Pipeline reinitialized")
print("💡 Note: Now using Brian2 dt=0.1ms (100 microseconds)")
print("✅ This eliminates duplicate spike conflicts")

In [ ]:
# CPU inference benchmark
print("⏱ CPU Inference Evaluación\n")

# Garantir que num_samples esteja definido (célula can ser executada isoladamente)
if 'num_samples' not in globals():
    num_samples = 100  # Reduzido of 1000 for speed
latencies_cpu = []

print(f"Processando {num_samples} amostras...")

for i in tqdm(range(num_samples), desc="CPU Inferences"):
    transaction = df_test.iloc[i].to_dict()

    try:
        start = time.perf_counter()
        prediction = pipeline.predict(transaction)
        end = time.perf_counter()

        latency_ms = (end - start) * 1000
        latencies_cpu.append(latency_ms)
    except ValueError as e:
        if "spike more than once" in str(e):
            print(f"\n error of colisão of spikes na amostra {i}!")
            print(f"Detalhes: {e}")
            print("Tentando continuar with a next amostra...")
            continue
        else:
            print(f"\n error inesperado na amostra {i}: {e}")
            raise e
    except Exception as e:
        print(f"\n error genérico na amostra {i}: {e}")
        raise e

# Estatísticas CPU
if latencies_cpu:
    cpu_metrics = {
        'mean_latencia_ms': np.mean(latencies_cpu),
        'median_latencia_ms': np.median(latencies_cpu),
        'p95_latencia_ms': np.percentile(latencies_cpu, 95),
        'p99_latencia_ms': np.percentile(latencies_cpu, 99),
        'rendimiento_fps': 1000 / np.mean(latencies_cpu),
        'total_time_s': sum(latencies_cpu) / 1000,
    }

    print("\n" + "="*50)
    print("CPU Evaluación RESULTS")
    print("="*50)
    print(f"Amostras processadas: {len(latencies_cpu)}/{num_samples}")
    print(f"latencia average: {cpu_metrics['mean_latencia_ms']:.2f} ms")
    print(f"latencia median: {cpu_metrics['median_latencia_ms']:.2f} ms")
    print(f"latencia P95: {cpu_metrics['p95_latencia_ms']:.2f} ms")
    print(f"latencia P99: {cpu_metrics['p99_latencia_ms']:.2f} ms")
    print(f"Rendimiento: {cpu_metrics['rendimiento_fps']:.1f} transactions/s")
    print(f"time total: {cpu_metrics['total_time_s']:.2f} s")
    print("="*50)
else:
    print("\n Nenhuma amostra processada with sucesso.")

## 2. Intel Loihi 2 Simulation

Now let's simulate what the performance would be on the Loihi 2 neuromórfico chip.

In [ ]:
# Inicializar Loihi simulator
loihi = LoihiSimulator()

# Network specifications
network_neurons = 256 + 128 + 64 + 2 # 450 neurons
network_synapses = (256 * 128) + (128 * 64) + (64 * 2) # 41,088 synapses

print(f"🧠 Neural Network:")
print(f"  - Neurons: {network_neurons:,}")
print(f"  - Synapses: {network_synapses:,}")
print(f"\n Loihi 2 Specs:")
print(f" - Cores: {loihi.specs.num_cores}")
print(f" - neurons: {loihi.specs.total_neurons:,}")
print(f" - potencia/core: {loihi.specs.potencia_per_core_active*1000:.1f} mW")

In [ ]:
# Execute Loihi Evaluación
print("\n⏳ Executing Loihi 2 Evaluación...\n")

# Ensure num_samples is available even if the CPU cell was not executed
if 'num_samples' not in locals():
    num_samples = 100
    print("⚠️ Advertencia: 'num_samples' not defined previously; using default of 100 inferences.")

loihi_metrics = loihi.benchmark_inference(
    network_neurons=network_neurons,
    network_synapses=network_synapses,
    num_inferences=num_samples,
    simulation_time_ms=100.0
)

print("\n" + "="*50)
print("LOIHI 2 Evaluación RESULTS")
print("="*50)
print(f"latencia: {loihi_metrics.latencia_ms:.2f} ms")
print(f"Rendimiento: {loihi_metrics.rendimiento_fps:.1f} transactions/s")
print(f"energía total: {loihi_metrics.energía_mj:.2f} mJ")
print(f"potencia average: {loihi_metrics.potencia_mw:.2f} mW")
print(f"Cores Usados: {loihi_metrics.cores_used}/{loihi.specs.num_cores}")
print(f"total of Spikes: {loihi_metrics.total_spikes:,}")
print(f"Ops Sinápticas: {loihi_metrics.synaptic_operations:,}")
print("="*50)

## 3. Comparison e Análisis

In [ ]:
# Comparar with CPU
cpu_power_w = 65.0 # TDP típico of CPU Intel Core i5/i7

comparison = compare_with_cpu(
 loihi_metrics=loihi_metrics,
 cpu_latencia_ms=cpu_metrics['mean_latencia_ms'],
 cpu_power_w=cpu_power_w
)

print("\n" + "="*50)
print("LOIHI vs CPU COMPARISON")
print("="*50)
print(f" Speedup: {comparison['speedup']:.2f}x")
print(f" potencia eficiencia: {comparison['potencia_eficiencia']:.2f}x")
print(f" energía eficiencia: {comparison['energía_eficiencia']:.2f}x")
print(f"\n Reductions:")
print(f" - latencia: {comparison['latencia_reduction_percent']:.1f}%")
print(f" - potencia: {comparison['potencia_reduction_percent']:.1f}%")
print(f" - energía: {comparison['energía_reduction_percent']:.1f}%")
print("="*50)

In [ ]:
# Crear DataFrame comparativo
comparison_df = pd.DataFrame({
 'metric': ['latencia (ms)', 'Rendimiento (TPS)', 'potencia (mW)', 'energía (mJ)'],
 'CPU': [
 cpu_metrics['mean_latencia_ms'],
 cpu_metrics['rendimiento_fps'],
 cpu_power_w * 1000, # 65W in mW
 (cpu_potencia_w * 1000 * cpu_metrics['mean_latencia_ms']) / 1000 # mJ
 ],
 'Loihi 2': [
 loihi_metrics.latency_ms,
 loihi_metrics.throughput_fps,
 loihi_metrics.power_mw,
 loihi_metrics.energy_mj
 ]
})

comparison_df['Improvement'] = comparison_df['CPU'] / comparison_df['Loihi 2']
comparison_df

## 4. Visualizacións

In [ ]:
# Plot 1: Comparison of latencia
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# latencia
ax1 = axes[0, 0]
latencies = [cpu_metrics['mean_latencia_ms'], loihi_metrics.latencia_ms]
labels = ['CPU', 'Loihi 2']
colors = ['#FF6B6B', '#4ECDC4']
ax1.bar(labels, latencies, color=colors, alpha=0.8, edgecolor='black')
ax1.set_ylabel('latencia (ms)', fontsize=12)
ax1.set_title('latencia per Inference', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(latencies):
 ax1.text(i, v + 0.5, f'{v:.2f}ms', ha='center', fontweight='bold')

# Rendimiento
ax2 = axes[0, 1]
rendimientos = [cpu_metrics['rendimiento_fps'], loihi_metrics.rendimiento_fps]
ax2.bar(labels, rendimientos, color=colors, alpha=0.8, edgecolor='black')
ax2.set_ylabel('Rendimiento (TPS)', fontsize=12)
ax2.set_title('Rendimiento (Transactions/second)', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
for i, v in enumerate(throughputs):
 ax2.text(i, v + 5, f'{v:.1f}', ha='center', fontweight='bold')

# potencia
ax3 = axes[1, 0]
powers = [cpu_power_w * 1000, loihi_metrics.power_mw]
ax3.bar(labels, potencias, color=colors, alpha=0.8, edgecolor='black')
ax3.set_ylabel('potencia (mW)', fontsize=12)
ax3.set_title('consumption of potencia', fontsize=14, fontweight='bold')
ax3.set_yscale('log')
ax3.grid(axis='y', alpha=0.3)
for i, v in enumerate(powers):
 ax3.text(i, v * 1.2, f'{v:.0f}mW', ha='center', fontweight='bold')

# energía by inference
ax4 = axes[1, 1]
cpu_energía = (cpu_potencia_w * 1000 * cpu_metrics['mean_latencia_ms']) / 1000
energies = [cpu_energy, loihi_metrics.energy_mj / num_samples]
ax4.bar(labels, energies, color=colors, alpha=0.8, edgecolor='black')
ax4.set_ylabel('energía (mJ)', fontsize=12)
ax4.set_title('energía per Inference', fontsize=14, fontweight='bold')
ax4.set_yscale('log')
ax4.grid(axis='y', alpha=0.3)
for i, v in enumerate(energies):
 ax4.text(i, v * 1.5, f'{v:.2f}mJ', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('hardware_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Gráficos salvos in 'hardware_comparison.png'")

In [ ]:
# Plot 2: eficiencia Gains
fig, ax = plt.subplots(figsize=(10, 6))

metrics_names = ['Speedup', 'potencia\nEfficiency', 'energía\nEfficiency']
improvements = [
 comparison['speedup'],
 comparison['potencia_eficiencia'],
 comparison['energía_eficiencia']
]

bars = ax.barh(metrics_names, improvements, color='#4ECDC4', alpha=0.8, edgecolor='black')
ax.set_xlabel('Fator of bestia (X vezes better)', fontsize=12)
ax.set_title('Loihi 2 eficiencia Gains vs CPU', fontsize=14, fontweight='bold')
ax.axvline(x=1, color='red', linestyle='--', linewidth=2, label='Baseline CPU')
ax.grid(axis='x', alpha=0.3)
ax.legend(fontsize=10)

for i, (bar, val) in enumerate(zip(bars, improvements)):
 ax.text(val + 50, i, f'{val:.1f}x', va='center', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig('eficiencia_gains.png', dpi=300, bbox_inches='tight')
plt.show()

print(" Gráficos salvos in 'eficiencia_gains.png'")

In [ ]:
# Plot 3: latencia distribution CPU
fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(latencies_cpu, bins=50, color='#FF6B6B', alpha=0.7, edgecolor='black')
ax.axvline(cpu_metrics['mean_latencia_ms'], color='red', linestyle='--', 
 linewidth=2, label=f"average: {cpu_metrics['mean_latencia_ms']:.2f}ms")
ax.axvline(cpu_metrics['median_latencia_ms'], color='orange', linestyle='--', 
 linewidth=2, label=f"median: {cpu_metrics['median_latencia_ms']:.2f}ms")
ax.axvline(loihi_metrics.latencia_ms, color='green', linestyle='-', 
 linewidth=3, label=f"Loihi: {loihi_metrics.latencia_ms:.2f}ms")

ax.set_xlabel('latencia (ms)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('latencia distribution - CPU vs Loihi 2', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('latencia_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(" Gráficos salvos in 'latencia_distribution.png'")

In [ ]:
# Plot 4: ROC Curve
from sklearn.metrics import roc_curve, auc

# Obter predictions probabilísticas of model
print("Generando predictions for ROC Curve...")
y_scores = []
for i in tqdm(range(len(X_test)), desc="Predictions"):
    transaction = df_test.iloc[i].to_dict()
    try:
        prediction = pipeline.predict(transaction)
        # Extrair o value numérico of prediction
        # pipeline.predict() can retornar dict, float ou int
        if isinstance(prediction, dict):
            # Se for dict, tentar extrair probability of fraude
            score = prediction.get('fraud_probability', prediction.get('prediction', 0.5))
        elif isinstance(prediction, (int, float)):
            score = float(prediction)
        else:
            score = 0.5
        y_scores.append(score)
    except Exception as e:
        print(f"\nErro na amostra {i}: {e}")
        y_scores.append(0.5)  # value pattern in case of error

# Converter for array numpy
y_scores = np.array(y_scores, dtype=float)

# Calcular ROC
fpr, tpr, thresholds = roc_curve(y_test_np, y_scores)
roc_auc = auc(fpr, tpr)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(fpr, tpr, color='#4ECDC4', linewidth=3, label=f'ROC Curve (AUC = {roc_auc:.3f})')
ax.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Random Classifier')
ax.fill_between(fpr, tpr, alpha=0.3, color='#4ECDC4')

ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curve - Fraud Detection Performance', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ ROC Curve salva in 'roc_curve.png' (AUC = {roc_auc:.3f})")

In [ ]:
# Plot 5: Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report

# Converter predictions to binary (threshold = 0.5)
# y_scores is already an array numpy of floats of célula anterior
y_pred = (y_scores > 0.5).astype(int)

# Calcular confusion matrix
cm = confusion_matrix(y_test_np, y_pred)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, 
            square=True, linewidths=2, linecolor='black',
            xticklabels=['Legítima', 'Fraude'],
            yticklabels=['Legítima', 'Fraude'],
            annot_kws={'size': 16, 'weight': 'bold'},
            ax=ax)

ax.set_xlabel('Prediction', fontsize=12, fontweight='bold')
ax.set_ylabel('value Real', fontsize=12, fontweight='bold')
ax.set_title('Confusion Matrix - Fraud Detection', fontsize=14, fontweight='bold')

# Add metrics no plot
tn, fp, fn, tp = cm.ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

metrics_text = f'Accuracy: {accuracy:.3f}\nPrecision: {precision:.3f}\nRecall: {recall:.3f}\nF1-Score: {f1:.3f}'
ax.text(1.15, 0.5, metrics_text, transform=ax.transAxes, 
        fontsize=11, verticalalignment='center',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Confusion Matrix salva in 'confusion_matrix.png'")
print(f"\n📊 Classification Metrics:")
print(f"  Accuracy:  {accuracy:.3f}")
print(f"  Precision: {precision:.3f}")
print(f"  Recall:    {recall:.3f}")
print(f"  F1-Score:  {f1:.3f}")

# Relatório detalhado
print("\n" + "="*50)
print("CLASSIFICATION REPORT")
print("="*50)
print(classification_report(y_test_np, y_pred, target_names=['Legítima', 'Fraude']))

## 5. Análisis of Escalabilidade

In [ ]:
# Simulate different transaction volumes
print("📈 Analyzing scalability...\n")

volumes = [100, 500, 1000, 5000, 10000, 50000]
scalability_results = []

for vol in tqdm(volumes, desc="Volumes"):
 # CPU: latencia grows linearly
 cpu_total_time_s = (cpu_metrics['mean_latencia_ms'] * vol) / 1000
 cpu_total_energy_mj = (cpu_power_w * 1000 * cpu_total_time_s)
 
 # Loihi: much better scalability
 loihi_sim = loihi.benchmark_inference(
 network_neurons=network_neurons,
 network_synapses=network_synapses,
 num_inferences=vol,
 simulation_time_ms=100.0
 )
 loihi_total_time_s = (loihi_sim.latency_ms * vol) / 1000
 
 scalability_results.append({
 'volume': vol,
 'cpu_time_s': cpu_total_time_s,
 'loihi_time_s': loihi_total_time_s,
 'cpu_energía_mj': cpu_total_energía_mj,
 'loihi_energía_mj': loihi_sim.energía_mj,
 'speedup': cpu_total_time_s / loihi_total_time_s
 })

scalability_df = pd.DataFrame(scalability_results)
scalability_df

In [ ]:
# Plot of escalabilidade
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# time of execution
ax1 = axes[0]
ax1.plot(scalability_df['volume'], scalability_df['cpu_time_s'], 
 marker='o', linewidth=2, markersize=8, label='CPU', color='#FF6B6B')
ax1.plot(scalability_df['volume'], scalability_df['loihi_time_s'], 
 marker='s', linewidth=2, markersize=8, label='Loihi 2', color='#4ECDC4')
ax1.set_xlabel('Volume of Transactions', fontsize=12)
ax1.set_ylabel('time total (s)', fontsize=12)
ax1.set_title('Escalabilidade: time of Execution', fontsize=14, fontweight='bold')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# energía consumida
ax2 = axes[1]
ax2.plot(scalability_df['volume'], scalability_df['cpu_energía_mj'], 
 marker='o', linewidth=2, markersize=8, label='CPU', color='#FF6B6B')
ax2.plot(scalability_df['volume'], scalability_df['loihi_energía_mj'], 
 marker='s', linewidth=2, markersize=8, label='Loihi 2', color='#4ECDC4')
ax2.set_xlabel('Volume of Transactions', fontsize=12)
ax2.set_ylabel('energía total (mJ)', fontsize=12)
ax2.set_title('Escalabilidade: energía consumption', fontsize=14, fontweight='bold')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('scalability_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(" Gráficos salvos in 'scalability_analysis.png'")

## 6. Conclusiones e Recommendations

### main Resultados

with base nos benchmarks realizados:

**Performance:**
- Loihi 2 oferece **speedup significativo** in latencia
- Rendimiento very superior for Processing paralelo
- consistent and predictable latencia (event-driven)

**eficiencia Energética:**
- **Drastic reduction** no energía consumption (1000x+)
- Ideal for edge computing e dispositivos móveis
- Viable continuous operation with battery

**Escalabilidade:**
- Vantagem of Loihi aumenta with volume of transactions
- Processing paralelo Native
- Multi-chip for applications very large

### Recommendations of Deployment

#### Use **CPU/GPU** when:
- Volume low of transactions (< 1000/s)
- Prototipagem e Development
- Code flexibility é priority
- Infraestrutura existente available

#### Use **Loihi 2** when:
- high volume of transactions (> 10000/s)
- eficiencia energética é crítica
- Edge computing / dispositivos móveis
- latencia ultra-low é requisito
- 24/7 Operation with restrictions of energía

### Próximos pasos

1. **Implementation real in Loihi**: Migrar of Brian2 for Lava
2. **Optimization of arquitetura**: Ajustar for maximizar eficiencia of Loihi
3. **Tests in Production**: Validar with tráfego real
4. **Custo-benefício**: Análisis of TCO (total Cost of Ownership)
5. **Comparison with outros chips**: TrueNorth, SpiNNaker, BrainScaleS

In [ ]:
# Guardar results in CSV
comparison_df.to_csv('hardware_benchmark_results.csv', index=False)
scalability_df.to_csv('scalability_results.csv', index=False)

print("\n Resultados salvos:")
print(" - hardware_benchmark_results.csv")
print(" - scalability_results.csv")
print(" - hardware_comparison.png")
print(" - eficiencia_gains.png")
print(" - latencia_distribution.png")
print(" - scalability_analysis.png")